# Точный и асимптотический критерий Вилкоксона
Применяется при сравнении двух выборок для непараметрической проверки гипотезы о равенстве средних в этих выборках.

*Точный критерий Вилкоксона* применяется к двум выборкам размера $n_1$ и $n_2$ соответственно. Предположение о равенстве средних заменяется эксвивалентным о равенстве числа инверсий $u$ и контринверсий $v$, т.е. $H_0: u = v$, $H_1: u > v$.

Сгенерируем данные для демонстрации точного критерия.

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
from scipy.stats import norm, rankdata
# import matplotlib.pyplot as plt
# from numpy.linalg import eig
# data = pd.read_table("wilcoxon.csv", decimal=",", sep=";", usecols=[0,1,2,4])

n1 = 20
n2 = 26
d1 = norm(1,2).rvs(n1)
d2 = norm(1,2).rvs(n2)
data1 = pd.DataFrame()
data1["val"] = d1
data1["group"] = 1
data2 = pd.DataFrame()
data2["val"] = d2
data2["group"] = 2
data = data1.append(data2)
del data1, data2

Точный критерий Вилкоксона подсчитывает число инверсий в данных и сравнивает его с критическим для данного объёма выборки числом инверсий.

In [2]:
# group: group to count inversions against
def count_inversions(data, group):
    sorteddata = data.sort("val")
    selector = sorteddata["group"] == group
    this = sorteddata[selector]
    other = sorteddata[np.logical_not(selector)]
    inv_count = 0
    for (this_index, u) in this.iterrows():
        inv_count += len(other[other.index > this_index])
    return inv_count

print "число инверсий в данных: " + str(count_inversions(data, 1))
print "объём данных: " + str(n1+ n2)

число инверсий в данных: 310
объём данных: 46


Имея таблицу распределения Вилкоксона (насколько близко к предельно допустимому для такого объёма данных находится данное число инверсий), по числу инверсий можно понять, отвергать или не отвергать гипотезу $H_0$.

Пользоваться таблицей неудобно, поэтому существует асимптотический критерий Вилкоксона (критерий Манна-Уитни). Так как известны математическое ожидание и дисперсия числа инверсий, мы можем (выразив для удобства число инверсий через сумму рангов $R$: $r_i$ -- число инверсий второй выборки с i-м элементом первой, $u = \sum_{i=1}^{n_1}(r_i - i) = R - \frac{n_1(n_1 + 1)}{2}$) воспользоваться центральной предельной теоремой и использовать асимптотическую статистику $$t = \frac{R - \frac{n_1(n_1 + 1)}{2} - \frac{n_1n_2}{2}}{\sqrt{\frac{n_1n_2(n_1+n_2+1)}{12}}} \sim N(0,1)$$

In [3]:
data["rank"] = rankdata(data.val)
R = data[data.group == 1]['rank'].sum()
t = (R - n1*(n1 + 1)/2 - n1*n2/2) / np.sqrt(n1*n2*(n1 + n2 + 1)/12)
critical_value = norm(0,1).ppf(0.975)
print "доверительный интервал: (%f, %f)"%(-critical_value, critical_value)
print "значение статистики:", t

доверительный интервал: (-1.959964, 1.959964)
значение статистики: 0.975132855791


Стандартная реализация этого теста:

In [4]:
t, pvalue = sp.stats.ranksums(data[data.group == 1].val, data[data.group == 2].val)
print "t =", t
print "p-value = ", pvalue

t = 0.974973246517
p-value =  0.329573530863


# Знаковый критерий и критерий Вилкоксона для зависимых выборок

In [5]:
data = pd.DataFrame()
n = 50
data["t1"] = norm(1,2).rvs(n)
data["t2"] = data["t1"]*2 + norm(0,2).rvs(n)

### Интерпретация данных
`data["t1"]` -- значения некоторой величины в $n$ экспериментах в момент времени $t_1$, `data["t2"]` -- значения той же величины в тех же экспериментах в момент времени $t_2$. По значениям этой величины можно сделать вывод $r_i\left(x_{t_1}, x_{t_2}\right)$ об успехе или неудаче $i$-го эксперимента. О <<направленности>> результата говорит знак $r_i\left(x_{t_1}, x_{t_2}\right)$: $<0$ в случае неудачи, $0$ в случае отсутствия результата, $>0$ в случае успешного исхода, абсолютное значение $r_i\left(x_{t_1}, x_{t_2}\right)$ несёт информацию о степени эффективности результата. Эффективность для различных $i$ должна быть сравнима.

В демонстрации будем использовать $\forall i\in 1:n\;\; r_i\left(x_{t_1}, x_{t_2}\right) = x_{t_2} - x_{t_1}$. 

## Критерий знаков
Определим статистики
$$ S_+ = \#\left\lbrace i\in 1:n \middle\vert r_i\left(x_{t_1}, x_{t_2}\right) > 0 \right\rbrace $$
$$ S_- = \#\left\lbrace i\in 1:n \middle\vert r_i\left(x_{t_1}, x_{t_2}\right) < 0 \right\rbrace $$

Сведём задачу к определению того, какова вероятность $p$ успеха эксперимента, то есть к проверке гипотезы $H_0: p = {1\over 2}$ против альтернативы  $H_1: p > {1\over 2}$. В случае истинности нулевой гипотезы число успехов $S_+ \sim \mathrm{Bin}(n, {1\over 2})$. Поэтому в случае $P(\xi >= S_+) = \sum_{k=t}^n \mathsf{C}_n^k \left({1\over 2}\right)^k < \alpha $, где $\alpha$ -- уровень значимости, гипотезу о незначительности результатов эксперимента можно отвергнуть.

In [6]:
res = data.t2 - data.t1
splus = len(res[res > 0])
t = sp.stats.binom(n, 0.5).sf(splus)
print "значение статистики:", t

значение статистики: 0.00330022398341


В нашем случае для всех стандартных уровней значимости гипотеза о незначимости внешнего воздействия между моментами времени $t_1$ и $t_2$ может быть отвергнута.

## Критерий Вилкоксона для зависимых выборок
Обозначим $R_i$ -- ранг результата $i$-го эксперимента в множестве всех абсолютных значений результатов экспериментов $\left\lbrace \left\vert r_i\left(x_{t_1}, x_{t_2}\right)\right\vert\right\rbrace_{i=1}^n$. Тогда случайная величина $W = \min\left\lbrace\sum_{i \in S_+} R_i, \sum_{i\in S_-} R_i\right\rbrace$ -- статистика Вилкоксона, для которой можно использовать нормальную аппроксимацию $$t = {W - EW \over \sqrt{DW}} = {W - {n(n+1)\over 4} \over \sqrt{n(n+1)(2n+1) \over 24}} \overset{\sim}{\to} N(0, 1)$$

*NB: Интересно, что в исходной [статье](http://sci2s.ugr.es/keel/pdf/algorithm/articulo/wilcoxon1945.pdf) упоминается именно эта статистика $W$, тогда как на [английский Википедии](https://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test#cite_ref-2) используется $W = \left\vert\sum_{i \in S_+} R_i - \sum_{i\in S_-} R_i\right\vert$, а на [MachineLearning.ru](http://www.machinelearning.ru/wiki/index.php?title=Критерий_Уилкоксона_для_связных_выборок) $W = \sum_{i \in S_+} R_i$. Все статистики показывают, насколько сильно смещение баланса в сторону исходов какого-либо одного типа, но в сравнении с разностью сумм рангов с Вики смысл статистик из оригинальной статьи и с Машинного Обучения кажется гораздо менее явным*

In [7]:
# cast to pandas.Series is to ease selection of positive and negative ranks
ranks = pd.Series(sp.stats.rankdata(np.abs(res)))
W = np.min([ranks[res > 0].sum(), ranks[res < 0].sum()])
print "минимальная сумма рангов:", W
t = (W - n*(n+1) / 4) / np.sqrt(n*(n+1)*(2*n+1)/24)
print "критическое значение: (%f, %f)"%(-critical_value, critical_value)
print "значение статистики:", t

минимальная сумма рангов: 379.0
критическое значение: (-1.959964, 1.959964)
значение статистики: -2.49057490896


Стандартная реализация метода:

In [8]:
T, pvalue = sp.stats.wilcoxon(data.t1, data.t2)
print T, pvalue

379.0 0.0125824951599


Таким образом, согласно знаковому критерию Вилкоксона медианы выборок из нормального распределения с математическим ожиданием 0 и 2 всё же отличаются, несмотря на большую дисперсию каждого из распределений.

- почему одинаковые наблюдения называются ties?
- почему [реализация](https://github.com/scipy/scipy/blob/v0.15.1/scipy/stats/stats.py#L4103) ранжированных сумм не содержит чего-то важного (вспомнить, чего!)

# Критерии однородности для двух и более выборок

Для $k>=2$ выборок понятие инверсии не обобщается однозначно, поэтому в исходном виде критерий Вилкоксона не подходит. 

Имея $k$ выборок $X^j = \left\lbrace x_i^j\right\rbrace_{i=1}^{n_j}, j\in 1:k$, каждая из распределения $\mathcal{P}_j$, проверим гипотезу о совпадении этих распределений $\mathcal{P}_1 = \ldots = \mathcal{P}_k$.